## Face Recognition


**Описание задачи**: В этой задаче требуется сделать модель распознавания лиц на основе фотографий людей. В ходе работы будут рассмотрены несколько моделей и несколько подходов к решению этой задачи.

In [1]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import datasets, models, transforms
import numpy as np
import os
from pathlib import Path
import torch.nn as nn
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import os
from torch.utils.data import Subset
from itertools import combinations
import random

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

В качестве датасета для решения этой задачи я буду использовать довольно популярный датасет для распознавания лиц: LFW Dataset. Его я взял с Kaggle

**Описание датасета**: в датасете представлены папки с фотографиями людей для каждого человека. У каждого человека 2< фотографий.

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/lfw-dataset


Чтобы не было утечек, надо сначала разбить папки на трейн и тест и потом делить на пары

In [4]:
folders = [name for name in os.listdir('/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled')]
train_classes, test_classes = train_test_split(
    folders,
    test_size=0.2,
    random_state=40,
    shuffle=True
)

In [5]:
folders = sorted(folders)
path = '/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
images_num = 0
for name in folders:
  files = os.listdir(f'/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{name}')
  image_files = [f for f in files if f.lower().endswith('.jpg')]
  images_num += len(image_files)

print("Number of images: ", images_num)

Number of images:  13233


Дальнейшее решение предполагает использование **contrastive learning** для обучени модели, поэтому я разобью изображения лиц положительные и негативные пары: положительные - один человек => метка 1, негативные - разные люди => метка 0.

ТРЕЙН ПАРЫ

In [6]:
dataset_path = "/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled"

people_images = {person: os.listdir(os.path.join(dataset_path, person))
                 for person in train_classes
                 if os.path.isdir(os.path.join(dataset_path, person))}

positive_pairs = []
negative_pairs = []

for person, images in people_images.items():
    if len(images) > 1:
        positive_pairs += [(i,1) for i in combinations(images, 2)]

people = list(people_images.keys())
for _ in range(len(positive_pairs)):
    p1, p2 = random.sample(people, 2)
    img1 = random.choice(people_images[p1])
    img2 = random.choice(people_images[p2])
    negative_pairs.append(((img1, img2),-1))

print(f"✔ Создано {len(positive_pairs)} позитивных тренировочных пар и {len(negative_pairs)} негативных пар.")
pairs_train = positive_pairs + negative_pairs

✔ Создано 230017 позитивных тренировочных пар и 230017 негативных пар.


ТЕСТ ПАРЫ

In [7]:
dataset_path = "/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled"

# Собираем список людей и их изображений
people_images = {person: os.listdir(os.path.join(dataset_path, person))
                 for person in test_classes
                 if os.path.isdir(os.path.join(dataset_path, person))}

positive_pairs = []
negative_pairs = []

for person, images in people_images.items():
    if len(images) > 1:
        positive_pairs += [(i,1) for i in combinations(images, 2)]

people = list(people_images.keys())
for _ in range(len(positive_pairs)):
    p1, p2 = random.sample(people, 2)
    img1 = random.choice(people_images[p1])
    img2 = random.choice(people_images[p2])
    negative_pairs.append(((img1, img2),-1))

print(f"✔ Создано {len(positive_pairs)} позитивных тестовых пар и {len(negative_pairs)} негативных пар.")
pairs_test = positive_pairs + negative_pairs

✔ Создано 12240 позитивных тестовых пар и 12240 негативных пар.


In [ ]:
n = Image.open('/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/AJ_Cook/AJ_Cook_0001.jpg')
print(n.size)

(250, 250)


Для дальнейшего обучения модели напишем кастомный датасет

In [8]:
class FaceEshkereDatasetPair(Dataset):
  def __init__(self, pairs, mode):
    self.pairs = pairs
    self.mode = mode
    self.transform_train = transforms.Compose([
      transforms.Resize((112, 112)),
      transforms.RandomHorizontalFlip(p=0.2),
      transforms.ToTensor()
      ])
    self.transform_test = transforms.Compose([
      transforms.Resize((112, 112)),
      transforms.ToTensor()
      ])
  def __len__(self):
    return len(self.pairs)

  def __getitem__(self, index):
    path_to_image0 = Path(f"/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{self.pairs[index][0][0][:-9]}/{self.pairs[index][0][0]}") #надо бы поменять конечно, но работает
    path_to_image1 = Path(f"/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled/{self.pairs[index][0][1][:-9]}/{self.pairs[index][0][1]}")

    label = self.pairs[index][1]

    if self.mode == 'train':
      image0 = self.transform_train(Image.open(path_to_image0))
      image1 = self.transform_train(Image.open(path_to_image1))
    if self.mode == 'test':
      image0 = self.transform_test(Image.open(path_to_image0))
      image1 = self.transform_test(Image.open(path_to_image1))

    return image0, image1, label


In [9]:
test_dataset = FaceEshkereDatasetPair(pairs_test, 'test')
train_dataset = FaceEshkereDatasetPair(pairs_train, 'train')

test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)


In [10]:
len(train_dataset)

460034

# Модели

В моем решении я буду рассматривать несколько моделей для решения этой задачи:

Resnet 18 - сверточная нейросеть с маленьким числом параметров.
VIT - визуальный трансформер, который использует внимание вместо сверток

# Рассмотрим и обучим Resnet18

Мой выбор пал именно на реснет18 из-за его компактной архитектуры, так как эта задача требует больших ресурсов для  вычислительных мощностей которыми я располагаю. Благодаря не такому большрому числу параметров можно обучить его *больше*

In [19]:
from torchvision.models import resnet18
from tqdm import tqdm, tqdm_notebook

model = resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = True
model.fc = nn.Linear(512,128, bias=True)

Загрузка весов (этот этап кода нужен уже после обучения для теста модели)

In [17]:
w = torch.load('/content/sample_data/model_weights_epoch_9.pth',map_location=torch.device('cpu') )
model.load_state_dict(w)

<All keys matched successfully>

Функция с тренировкой модели и валидацией

In [13]:
import torch
from tqdm import tqdm
from torch.nn import DataParallel

def train(model, optimizer, criterion, epochs, train_dataloader, test_dataloader):
    summary_loss_train = []
    summary_loss_test = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs") # штука для 2 и более гпу
        model = DataParallel(model)

    for epoch in tqdm(range(epochs), desc="Epochs", ncols=100):
        train_loss = 0
        train_norm_variable = 0
        model.train()
        for image0, image1, label in train_dataloader:
            image0 = image0.to(device)
            image1 = image1.to(device)
            label = label.to(device)
            optimizer.zero_grad()

            embed0 = model(image0)
            embed1 = model(image1)

            loss = criterion(embed0, embed1, label)
            summary_loss_train.append(loss.item())
            train_loss += loss.item() * image0.size(0)
            train_norm_variable += image0.size(0)
            loss.backward()
            optimizer.step()

        print(f'Train loss: {train_loss/train_norm_variable}')

        torch.save(model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict(),
                   f'/kaggle/working/model_weights_epoch_{epoch + 14}.pth')
        print(f"Weights saved for epoch {epoch + 14}")

        test_loss = 0
        test_norm_variable = 0
        model.eval()

        with torch.no_grad():
            for image0, image1, label in test_dataloader:
                image0 = image0.to(device)
                image1 = image1.to(device)
                label = label.to(device)

                embed0 = model(image0)
                embed1 = model(image1)
                loss = criterion(embed0, embed1, label)
                summary_loss_test.append(loss.item())

                test_loss += loss.item() * image0.size(0)
                test_norm_variable += image0.size(0)

        print(f'Test loss: {test_loss/test_norm_variable}')
    return summary_loss_train, summary_loss_test

В качестве функции потерь я буду использовать **CosineEmbeddingLoss** из torch. Это как раз таки пример контрастного обучения на парах, где мы учим модель в векторном пространстве в котором у эмбедингов одних лиц косинусное расстояние больше (угол меньше), а у разных лиц наоборот.
Так же активно используются **TripletLoss** **ArcFaceLoss** и другие. Триплетный лосс похож на тот который я использую но только на одной итерации ошибка складывается сразу как с негативной так и с положительной парой а не поочередно.

In [14]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CosineEmbeddingLoss()

In [ ]:
train(model, optimizer, criterion, 15, train_dataloader, test_dataloader)

Using 2 GPUs


Epochs:   0%|                                                                | 0/15 [00:00<?, ?it/s]

Train loss: 0.0010565760603973883
Weights saved for epoch 14


Epochs:   7%|███▌                                                 | 1/15 [07:33<1:45:44, 453.15s/it]

Test loss: 1.034893308367048e-06
Train loss: 6.940654346180964e-07
Weights saved for epoch 15


...

Epochs:  93%|████████████████████████████████████████████████▌   | 14/15 [1:49:13<07:49, 469.63s/it]

Test loss: 1.3973031717081799e-09
Train loss: 7.952962600857713e-10
Weights saved for epoch 28


Epochs: 100%|████████████████████████████████████████████████████| 15/15 [1:56:47<00:00, 467.19s/it]

Test loss: -1.1605875832313269e-09


([0.4927073121070862,
  0.07531451433897018,
  0.006135598290711641,
  0.0010899826884269714,
  0.0004211678169667721,
  0.00019638286903500557,
  0.00010436214506626129,
  6.893370300531387e-05,
  6.0974154621362686e-05,
  4.7711655497550964e-05,
  3.4275464713573456e-05,
  3.372738137841225e-05,
  2.6932917535305023e-05,
  2.58483923971653e-05,
  2.366025000810623e-05,
  2.2020190954208374e-05,
  2.0219944417476654e-05,
  1.849886029958725e-05,
  2.0417850464582443e-05,
  1.6530510038137436e-05,
  1.7116311937570572e-05,
  1.5332363545894623e-05,
  1.5101395547389984e-05,
  1.3976357877254486e-05,
  1.2835022062063217e-05,
  1.4459248632192612e-05,
  1.3607088476419449e-05,
  1.2668780982494354e-05,
  1.2064352631568909e-05,
  1.3669021427631378e-05,
  9.695533663034439e-06,
  1.2082047760486603e-05,
  1.1049211025238037e-05,
  1.0640360414981842e-05,
  9.4524584710598e-06,
  1.0329298675060272e-05,
  9.403098374605179e-06,
  9.443610906600952e-06,
  9.323004633188248e-06,
  9.084120

In [28]:
import torch
import torch.nn.functional as F

# Убедимся, что CUDA доступен
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model.eval()
model.to(device)
threshold = 0.5  # Порог можно подобрать на валидационной выборке

# Будем хранить тензоры на GPU для вычисления accuracy
all_predictions = torch.tensor([], device=device)
all_labels = torch.tensor([], device=device)
c = 0
with torch.no_grad():
    for img1, img2, labels in test_loader:
        c += 1
        # Переносим данные на GPU
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

        # Получаем эмбеддинги
        emb1 = model(img1)
        emb2 = model(img2)

        # Нормализуем эмбеддинги
        emb1 = F.normalize(emb1, p=2, dim=1)
        emb2 = F.normalize(emb2, p=2, dim=1)

        # Вычисляем косинусную схожесть
        cos_sim = F.cosine_similarity(emb1, emb2)

        # Преобразуем схожесть в предсказания
        predictions = torch.where(cos_sim > threshold, 1, -1)

        # Сохраняем на GPU (избегаем передачи CPU)
        all_predictions = torch.cat([all_predictions, predictions])
        all_labels = torch.cat([all_labels, labels])
        if c == 1000:
            break

# Вычисляем accuracy на GPU
correct = (all_predictions == all_labels).float().sum()
accuracy = correct / len(all_labels)

# Переносим результат на CPU для вывода
print(f"Accuracy: {accuracy.cpu().item():.4f}")

Using device: cuda
Accuracy: 0.7569


**Вывод:** модель имеет accuracy 0.56 после 9 эпох обучения всех слоев. Это может говорить о том, что архитектура Resnet18 является слишком простой для этой задачи и свертке тяжело дается выделить патерны для лиц.
Однако обучение показало лучшие результаты, чем просто модель обученная на ImageNet (0.5 accuracy). Возможно стоит обучить модель на большее количество эпох, однако это крайне проблематично в силу вычислительных ресурсов в моем распоряжении.

# VIT трансформер

Я возьму пример **визуального трансформера** с HuggingFace.

По предпололжению, он должен работать лучше в силу внимания.
Так же это довольно массивная модель поэтому я буду файнтюнить только последний слой, к тому же модель файнтюнилась на задаче распознавания лиц но на другом датасете.

In [23]:
from huggingface_hub import login
import timm

token = "YOUR_TOKEN"

#авторизация
login(token)

model = timm.create_model("hf_hub:gaunernst/vit_tiny_patch8_112.arcface_ms1mv3", pretrained=True)


In [ ]:
w = torch.load('/content/sample_data/model_weights_epoch_9.pth',map_location=torch.device('cpu') )
model.load_state_dict(w)

Обучение:

In [ ]:
from torch.nn import DataParallel

def train(model, optimizer, criterion, epochs, train_dataloader, test_dataloader):
    summary_loss_train = []
    summary_loss_test = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs") # штука для 2 и более гпу
        model = DataParallel(model)

    for epoch in tqdm(range(epochs), desc="Epochs", ncols=100):
        train_loss = 0
        train_norm_variable = 0
        model.train()
        for image0, image1, label in train_dataloader:
            image0 = image0.to(device)
            image1 = image1.to(device)
            label = label.to(device)
            optimizer.zero_grad()

            embed0 = model(image0)
            embed1 = model(image1)

            loss = criterion(embed0, embed1, label)
            summary_loss_train.append(loss.item())
            train_loss += loss.item() * image0.size(0)
            train_norm_variable += image0.size(0)
            loss.backward()
            optimizer.step()

        print(f'Train loss: {train_loss/train_norm_variable}')

        torch.save(model.module.state_dict() if isinstance(model, DataParallel) else model.state_dict(),
                   f'/kaggle/working/model_weights_epoch_{epoch + 14}.pth')
        print(f"Weights saved for epoch {epoch + 14}")

        test_loss = 0
        test_norm_variable = 0
        model.eval()

        with torch.no_grad():
            for image0, image1, label in test_dataloader:
                image0 = image0.to(device)
                image1 = image1.to(device)
                label = label.to(device)

                embed0 = model(image0)
                embed1 = model(image1)
                loss = criterion(embed0, embed1, label)
                summary_loss_test.append(loss.item())

                test_loss += loss.item() * image0.size(0)
                test_norm_variable += image0.size(0)

        print(f'Test loss: {test_loss/test_norm_variable}')
    return summary_loss_train, summary_loss_test

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CosineEmbeddingLoss()

In [ ]:
train(model, optimizer, criterion, 15, train_dataloader, test_dataloader)

**Вывод:** модель показывает результаты значительно лучше. Я могу предроложить, что это связано с вниманием, так как моделт учит не локальный паттерн, а общую взаимосвязь патчей во всем изображении. 0.75

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model.eval()
model.to(device)
threshold = 0.5


all_predictions = torch.tensor([], device=device)
all_labels = torch.tensor([], device=device)
c = 0
with torch.no_grad():
    for img1, img2, labels in test_loader:
        c += 1
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        emb1 = model(img1)
        emb2 = model(img2)

        emb1 = F.normalize(emb1, p=2, dim=1)
        emb2 = F.normalize(emb2, p=2, dim=1)


        cos_sim = F.cosine_similarity(emb1, emb2)

        predictions = torch.where(cos_sim > threshold, 1, -1)

        all_predictions = torch.cat([all_predictions, predictions])
        all_labels = torch.cat([all_labels, labels])
        if c == 1000:
            break

correct = (all_predictions == all_labels).float().sum()
accuracy = correct / len(all_labels)
print(f"Accuracy: {accuracy.cpu().item():.4f}")

Дальнейший код демонстрирует как модель "видит" лица в векторном пространстве и какой между нимим будет угол. Можно пронаблюдать, что действительно, у одних лиц косинусная схожесть высока, а у разных маленькая

In [ ]:
def compare_images(
    model,          # Ваша модель, возвращающая эмбеддинги
    image_path1,    # Путь к первому изображению
    image_path2,    # Путь ко второму изображению
    transform=None, # Трансформы для изображений (если None - будут созданы автоматически)
    device='cuda' if torch.cuda.is_available() else 'cpu',
    similarity_threshold=0.5  # Порог для принятия решения
):
    """
    Сравнивает два изображения с помощью модели.
    Возвращает:
    - similarity_score (float): косинусная схожесть между эмбеддингами (от -1 до 1)
    - is_similar (bool): True если схожесть выше порога
    """

    # 1. Подготовка модели
    model = model.to(device)
    model.eval()

    # 2. Создание трансформов (если не переданы)
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize((112, 112)),  # Размер, ожидаемый моделью
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    # 3. Загрузка и преобразование изображений
    def load_image(path):
        img = Image.open(path).convert('RGB')
        return transform(img).unsqueeze(0).to(device)  # Добавляем batch-размерность

    img1 = load_image(image_path1)
    img2 = load_image(image_path2)

    # 4. Получение эмбеддингов
    with torch.no_grad():
        emb1 = model(img1)
        emb2 = model(img2)

    # 5. Нормализация и расчет схожести
    emb1 = F.normalize(emb1, p=2, dim=1)
    emb2 = F.normalize(emb2, p=2, dim=1)

    similarity = F.cosine_similarity(emb1, emb2).item()  # Извлекаем скалярное значение

    # 6. Принятие решения
    is_similar = similarity > similarity_threshold

    return {
        'similarity_score': similarity,
        'is_similar': is_similar,
        'threshold': similarity_threshold
    }
image_path1 = '/kaggle/input/kryptonit-train/train/images/000003/5.jpg'
image_path2 = '/kaggle/input/kryptonit-train/train/images/000003/1.jpg'
ans = compare_images(
    model,          # Ваша модель, возвращающая эмбеддинги
    image_path1,    # Путь к первому изображению
    image_path2,    # Путь ко второму изображению
    transform=None, # Трансформы для изображений (если None - будут созданы автоматически)
    device='cuda' if torch.cuda.is_available() else 'cpu',
    similarity_threshold=0.5  # Порог для принятия решения
)
print(ans)

{'similarity_score': 0.01819482445716858, 'is_similar': False, 'threshold': 0.5}


## Общий вывод по задаче

Исходя из результатов полученных из валидации моделей я могу сделать вывод что attention лучше работает в этой задаче чем свертки. Так же стоит учесть что эта задача требует gpu и большего количества эпох для обучения, в качестве развития этой задачи можно попробовать обучить эти модели на более мощных ресурсах